In [1]:
import os              
os.environ['PYTHONHASHSEED'] = '0'
import pandas as pd                                                    
import numpy as np                                                     
import scanpy as sc                                                                                 
from time import time                                                       
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from anndata import AnnData, read_h5ad, concat
from tqdm import tqdm
import scipy
import scipy.stats as ss

Results are based on the files generated by "fig4E_EDfig11c_gene_updown.ipynb"

In [4]:
def extract_gw(name):
    return name[:4]

def var_df(gw_result):
    # gw_ind = np.array(list(map(extract_gw, result.index)))
    # gw_result = result[gw_ind == gw]
    gw_var = pd.DataFrame(np.var(gw_result, axis=0))
    gw_var.columns = ['var']
    gw_var = gw_var.sort_values(by=['var'], ascending=False)
    # gw_var['rank'] = range(gw_var.shape[0])
    return gw_var

def var_gw(result_nz, result_zs):
    # result_nz = pd.read_csv(f"result/{cluster}_cluster_nz.csv", index_col=0)
    # result_zs = pd.read_csv(f"result/{cluster}_cluster_zs.csv", index_col=0)
    # gw_ind = np.array(list(map(extract_gw, result_nz.index)))
    var_nz = var_df(result_nz)
    var_nz.columns = ['nz_var']
    var_nz['nz_rank'] = range(var_nz.shape[0])
    var_zs = var_df(result_zs)
    var_zs.columns = ['zs_var']
    var_zs['zs_rank'] = range(var_zs.shape[0])
    var_nz = var_nz.join(var_zs)
    var_nz['rank'] = var_nz['nz_rank'] + var_nz['zs_rank']
    var_nz = var_nz[['nz_var', 'zs_var', 'nz_rank', 'zs_rank', 'rank']]
    var_nz = var_nz.sort_values(by=['rank'], ascending=True)
    return var_nz

def var_tot(cluster):
    gw_lst = ["GW15", "GW20", "GW22", "GW34"]
    result_nz = pd.read_csv(f"result/{cluster}_cluster_nz.csv", index_col=0)
    result_zs = pd.read_csv(f"result/{cluster}_cluster_zs.csv", index_col=0)
    
    gene_var_tot = []
    gw_name = []
    for gw in gw_lst:
        gw_ind = np.array(list(map(extract_gw, result_nz.index)))
        if np.sum(gw_ind == gw) > 1:
            result_nz_gw = result_nz[gw_ind == gw]
            result_zs_gw = result_zs[gw_ind == gw]
            var_nz = var_gw(result_nz_gw, result_zs_gw)
            gene_var = var_nz.iloc[:10, :].index
            gene_var_tot.append(gene_var)
            gw_name.append(gw.lower())
            # nz_gene = result_nz[gene_var]
            # zs_gene = result_zs[gene_var]
            # nz_gene.to_csv(f"result_var/{cluster}_{gw}_nz.csv")
            # zs_gene.to_csv(f"result_var/{cluster}_{gw}_zs.csv")
    gene_var_tot = np.array(gene_var_tot)
    gene_var_tot = pd.DataFrame(gene_var_tot)
    gene_var_tot.index = gw_name
    gene_var_tot = gene_var_tot.T 
    gene_var_tot.to_csv(f"result/gene_var/{cluster}.csv")
    # return gene_var_tot.T

In [115]:
os.makedirs("result/gene_var/", exist_ok=True)

cluster_lst = ["EN-ET-L5|6", "EN-ET-SP|L6b", "EN-IT-L2", "EN-IT-L3", "EN-IT-L4", "EN-IT-L5|6"]
for cluster in cluster_lst:
    var_tot(cluster)

In [126]:
def var_tot_df(cluster):
    gw_lst = ["GW15", "GW20", "GW22", "GW34"]
    result_nz = pd.read_csv(f"result/{cluster}_cluster_nz.csv", index_col=0)
    result_zs = pd.read_csv(f"result/{cluster}_cluster_zs.csv", index_col=0)
    
    # gene_var_tot = []
    # gw_name = []
    result_df = []
    for gw in gw_lst:
        gw_ind = np.array(list(map(extract_gw, result_nz.index)))
        if np.sum(gw_ind == gw) > 1:
            result_nz_gw = result_nz[gw_ind == gw]
            result_zs_gw = result_zs[gw_ind == gw]
            var_nz = var_gw(result_nz_gw, result_zs_gw)
            gene_var = var_nz.iloc[:10, :]
            gene_var['gene'] = gene_var.index
            gene_var['gw'] = gw.lower()
            gene_var['cluster'] = cluster
            gene_var = gene_var[['cluster', 'gw', 'gene', 'zs_var', 'nz_var']]
            result_df.append(gene_var)
    # gene_var_tot = np.array(gene_var_tot)
    # gene_var_tot = pd.DataFrame(gene_var_tot)
    # gene_var_tot.index = gw_name
    # gene_var_tot = gene_var_tot.T 
    # gene_var_tot.to_csv(f"result/gene_var/{cluster}.csv")
    return result_df

In [131]:
cluster_lst = ["EN-ET-L5|6", "EN-ET-SP|L6b", "EN-IT-L2", "EN-IT-L3", "EN-IT-L4", "EN-IT-L5|6"]
result_tot = []

for cluster in cluster_lst:
    result = var_tot_df(cluster)
    result = pd.concat(result, axis=0)
    # result = result.reset_index(drop=True)
    result_tot.append(result)
result_tot = pd.concat(result_tot, axis=0)
result_tot = result_tot.reset_index(drop=True)

/var/folders/mc/kqfjr86j5gz9cdyb9w1kfhn40000gp/T/ipykernel_70325/1104988266.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_var['gene'] = gene_var.index
/var/folders/mc/kqfjr86j5gz9cdyb9w1kfhn40000gp/T/ipykernel_70325/1104988266.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_var['gw'] = gw.lower()
/var/folders/mc/kqfjr86j5gz9cdyb9w1kfhn40000gp/T/ipykernel_70325/1104988266.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [135]:
result_tot.columns = ['group', 'gw', 'gene', 'expr_var', 'prop_var']
result_tot.to_csv("result/gene_var/genevar_result.csv")

In [2]:
result_tot = pd.read_csv("result/gene_var/genevar_result.csv", index_col=0)
result_tot

,group,gw,gene,expr_var,prop_var
0,EN-ET-L5|6,gw15,HCRTR2,5.295277,0.201010
1,EN-ET-L5|6,gw15,SEMA3E,0.279993,0.036000
2,EN-ET-L5|6,gw15,KLHL1,0.244956,0.026637
3,EN-ET-L5|6,gw15,TRPM3,0.178769,0.023137
4,EN-ET-L5|6,gw15,NR4A2,0.248539,0.020209
...,...,...,...,...,...
205,EN-IT-L5|6,gw34,ADRA2A,0.578819,0.068540
206,EN-IT-L5|6,gw34,TOX,0.643163,0.052979
207,EN-IT-L5|6,gw34,NR4A2,0.972648,0.047293
208,EN-IT-L5|6,gw34,KIAA0319,0.600913,0.052227


In [9]:
def var_full(cluster):
    gw_lst = ["GW15", "GW20", "GW22", "GW34"]
    # cluster = "EN-ET-L5|6"
    result_nz = pd.read_csv(f"result/{cluster}_cluster_nz.csv", index_col=0)
    result_zs = pd.read_csv(f"result/{cluster}_cluster_zs.csv", index_col=0)

    # gene_var_tot = []
    # gw_name = []
    var_dict = {}
    for gw in gw_lst:
        gw_ind = np.array(list(map(extract_gw, result_nz.index)))
        if np.sum(gw_ind == gw) > 1:
            result_nz_gw = result_nz[gw_ind == gw]
            result_zs_gw = result_zs[gw_ind == gw]
            var_nz = var_gw(result_nz_gw, result_zs_gw)
            var_dict[gw] = var_nz
    return var_dict

In [117]:
result_tot

,group,gw,gene,expr_var,prop_var
0,EN-ET-L5|6,gw15,HCRTR2,5.295277,0.201010
1,EN-ET-L5|6,gw15,SEMA3E,0.279993,0.036000
2,EN-ET-L5|6,gw15,KLHL1,0.244956,0.026637
3,EN-ET-L5|6,gw15,TRPM3,0.178769,0.023137
4,EN-ET-L5|6,gw15,NR4A2,0.248539,0.020209
...,...,...,...,...,...
205,EN-IT-L5|6,gw34,ADRA2A,0.578819,0.068540
206,EN-IT-L5|6,gw34,TOX,0.643163,0.052979
207,EN-IT-L5|6,gw34,NR4A2,0.972648,0.047293
208,EN-IT-L5|6,gw34,KIAA0319,0.600913,0.052227


In [187]:
# cluster="EN-ET-L5|6"
def var_generate(cluster, thre = 0.5):
    var_cluster = var_full(cluster)
    # var_cluster
    gene_cluster = np.unique(result_tot[result_tot['expr_var'] > thre]['gene'])
    
    gw_lst = list(var_cluster.keys())
    df_lst = []
    for gw in gw_lst:
        df = var_cluster[gw]
        df = df[df.index.isin(gene_cluster)][['zs_var']]
        df.columns = [f"{gw}"]
        df_lst.append(df)

    refine = pd.concat(df_lst, axis=1)#.loc[result_tot[result_tot['group'] == cluster]['gene'].values, :].drop_duplicates()
    
    refine['max_col'] = refine.idxmax(axis=1)
    refine = refine.sort_values(by=['max_col'])
    # gw_lst = ["GW15", "GW20", "GW22", "GW34"]
    refine_lst = []
    for gw in gw_lst:
        refine_gw = refine[refine['max_col'] == gw]
        if cluster != "EN-IT-L2":
            if gw == "GW15" or gw == "GW20":
                refine_gw = refine_gw.sort_values(by=gw, ascending = False)
            else:
                refine_gw = refine_gw.sort_values(by=gw, ascending = True)
        else:
            if gw == "GW22":
                refine_gw = refine_gw.sort_values(by=gw, ascending = False)
            else:
                refine_gw = refine_gw.sort_values(by=gw, ascending = True)
        refine_lst.append(refine_gw)
    refine_lst = pd.concat(refine_lst, axis=0)
    refine_lst = refine_lst.drop(['max_col'], axis=1)
    col_names = np.array(refine_lst.columns)
    col_names = cluster + " " + col_names
    refine_lst.columns = col_names
    refine_lst[f'{cluster} Yes'] = np.isin(refine_lst.index, result_tot[(result_tot['group'] == cluster) & (result_tot['expr_var'] > thre)]['gene'].values)
    return refine_lst

In [234]:
new_cmap = sns.color_palette("Blues",12)

def plot_heatmap(refine_lst, cluster, height = 14, thre = 0.5):
    fig = plt.figure(figsize=(10, height)) 
    vmax_val = np.quantile(refine_lst, 0.99)
    ax = sns.heatmap(refine_lst, cmap=new_cmap, linewidth=0.5, vmax=vmax_val, cbar_kws={"shrink": 0.3, "label": "Expression variance"},linecolor="white",square=True)      ## 
    ax.set_xlabel("Time points", fontdict={'fontsize': 12})
    ax.set_ylabel("Genes", fontdict={'fontsize': 12})
    fig.subplots_adjust(right=0.8, top=0.98)       ## top=1.05, 
    fig.savefig(f"fig_vargene/{cluster}_heatmap_{thre}.pdf", format = "pdf", dpi=500)
    plt.close()

In [246]:
thre = 0.8
# cluster_lst = ["EN-ET-L5|6", "EN-ET-SP|L6b", "EN-IT-L2", "EN-IT-L3", "EN-IT-L4", "EN-IT-L5|6"]
cluster_lst = ["EN-ET-SP|L6b", 'EN-ET-L5|6', "EN-IT-L5|6", "EN-IT-L4", "EN-IT-L3", "EN-IT-L2"]
refine_tot_raw = []
for cluster in cluster_lst:
    refine_lst = var_generate(cluster, thre=thre)
    refine_tot_raw.append(refine_lst)
refine_tot = pd.concat(refine_tot_raw, axis=1)

# col_order = ['EN-ET-L5|6 Yes', "EN-ET-SP|L6b Yes", "EN-IT-L2 Yes", "EN-IT-L3 Yes", "EN-IT-L4 Yes", "EN-IT-L5|6 Yes"]
col_order = ["EN-ET-SP|L6b Yes", 'EN-ET-L5|6 Yes', "EN-IT-L5|6 Yes", "EN-IT-L4 Yes", "EN-IT-L3 Yes", "EN-IT-L2 Yes"]
gene_order = []
for i in range(len(col_order)):
    gene_order.extend(list(refine_tot_raw[i][refine_tot_raw[i][col_order[i]]].index))
# gene_order = np.unique(gene_order)
refine_tot = refine_tot.loc[gene_order, :].drop_duplicates()
refine_tot = refine_tot.drop(col_order, axis=1)

In [247]:
if thre == 0.5:
    height = 14
elif thre == 0.8:
    height = 12
elif thre == 1:
    height = 10
plot_heatmap(refine_tot, cluster = "total", height=height, thre=thre)